In [ ]:
# ============================== IMPORTS ==============================
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
from twilio.rest import Client
from flask import Flask, jsonify
from google.colab import files

# ============================== UPLOAD FILE ==============================
uploaded = files.upload()  # Upload 'thermostat_logs.json'

# ============================== OPTIONAL: ENV VAR SETUP FOR COLAB ==============================
# 🛑 Replace these with your actual values
os.environ["TWILIO_ACCOUNT_SID"] = "twilio account sid"
os.environ["TWILIO_AUTH_TOKEN"] = "token"
os.environ["TWILIO_PHONE_NUMBER"] = "number"
os.environ["TO_PHONE_NUMBER"] = "number"

os.environ["EMAIL_SENDER"] = "email"
os.environ["EMAIL_PASSWORD"] = "app password"  # App password, not Gmail login
os.environ["EMAIL_RECEIVER"] = "reciver email"

# ============================== GLOBAL CONFIG ==============================
LOG_FILE = "thermostat_logs.json"

# ============================== CLASS: FORENSICS ==============================
class SmartThermostatForensics:
    def __init__(self, data_file):
        self.data_file = data_file
        self.data = None
        self.load_data()

    def load_data(self):
        try:
            with open(self.data_file, 'r') as file:
                self.data = json.load(file)
        except FileNotFoundError:
            print(f"Error: File {self.data_file} not found.")
            self.data = {}
        except json.JSONDecodeError:
            print(f"Error: File {self.data_file} is not valid JSON.")
            self.data = {}

    def analyze_temperature_history(self):
        print("\n📊 Analyzing Temperature History:")
        for entry in self.data.get('temperature_history', []):
            try:
                timestamp = datetime.utcfromtimestamp(entry['timestamp'])
                print(f"At {timestamp}, Temperature: {entry['temperature']}°C")
            except Exception as e:
                print(f"Error: {e}")

    def analyze_schedule(self):
        print("\n📅 Analyzing Schedule:")
        for day, periods in self.data.get('schedule', {}).items():
            print(f"{day}:")
            for p in periods:
                print(f"  From {p['start']} to {p['end']}, Set temperature: {p['temperature']}°C")

    def analyze_user_interactions(self):
        print("\n🧑‍💻 Analyzing User Interactions:")
        for interaction in self.data.get('user_interactions', []):
            try:
                timestamp = datetime.utcfromtimestamp(interaction['timestamp'])
                print(f"At {timestamp}, Action: {interaction['action']}")
            except Exception as e:
                print(f"Error: {e}")

    def analyze_network_traffic(self):
        print("\n🌐 Analyzing Network Traffic:")
        for event in self.data.get('network_traffic', []):
            print(f"Network event: {event}")

# ============================== LOG EXTRACTION ==============================
def extract_thermostat_logs(log_file):
    if not os.path.exists(log_file):
        print("❌ Log file not found!")
        return None
    with open(log_file, 'r') as file:
        logs = json.load(file)
    return pd.DataFrame(logs)

# ============================== ANALYSIS ==============================
def analyze_thermostat_logs(df):
    if df is None or df.empty:
        print("⚠ No logs to analyze.")
        return
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    plt.figure(figsize=(10,5))
    sns.lineplot(data=df, x="timestamp", y="temperature", label="Temperature")
    sns.lineplot(data=df, x="timestamp", y="setpoint", label="Setpoint")
    plt.title("Thermostat Temperature Trends")
    plt.xlabel("Timestamp")
    plt.ylabel("Temperature (°C)")
    plt.legend()
    plt.grid(True)
    plt.show()

# ============================== ANOMALY DETECTION ==============================
def detect_anomalies(df, send_alerts=True):
    if df is None or df.empty:
        print("⚠ No data for anomaly detection.")
        return None

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    model = IsolationForest(contamination=0.05)
    df['anomaly_score'] = model.fit_predict(df[['temperature', 'setpoint']])
    anomalies = df[df['anomaly_score'] == -1]

    if not anomalies.empty:
        print("\n🚨 Alert! Anomalies detected:")
        print(anomalies)

        alert_msg = f"🚨 {len(anomalies)} anomaly/anomalies detected in thermostat behavior!"
        if send_alerts:
            send_sms_alert(alert_msg)
            send_email_alert("Thermostat Security Alert", alert_msg)

    return anomalies

# ============================== ALERT SYSTEM ==============================
def send_sms_alert(message):
    try:
        client = Client(
            os.getenv("TWILIO_ACCOUNT_SID"),
            os.getenv("TWILIO_AUTH_TOKEN")
        )
        msg = client.messages.create(
            body=message,
            from_=os.getenv("TWILIO_PHONE_NUMBER"),
            to=os.getenv("TO_PHONE_NUMBER")
        )
        print(f"✅ SMS Sent! SID: {msg.sid}")
    except Exception as e:
        print(f"❌ SMS failed: {e}")

def send_email_alert(subject, message):
    try:
        msg = MIMEMultipart()
        msg['From'] = os.getenv("EMAIL_SENDER")
        msg['To'] = os.getenv("EMAIL_RECEIVER")
        msg['Subject'] = subject
        msg.attach(MIMEText(message, 'plain'))

        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(os.getenv("EMAIL_SENDER"), os.getenv("EMAIL_PASSWORD"))
        server.sendmail(os.getenv("EMAIL_SENDER"), os.getenv("EMAIL_RECEIVER"), msg.as_string())
        server.quit()
        print("✅ Email Sent Successfully!")
    except Exception as e:
        print(f"❌ Email failed: {e}")

# ============================== REPORTING ==============================
def generate_forensic_report(anomalies, report_file="forensic_report.txt"):
    with open(report_file, "w") as report:
        report.write("=== SMART THERMOSTAT FORENSIC REPORT ===\n")
        report.write(f"Total Anomalies Detected: {len(anomalies)}\n\n")
        report.write("Anomaly Details:\n")
        report.write(anomalies.to_string(index=False))
    print(f"📄 Report saved as: {report_file}")

def generate_forensic_timeline(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    plt.figure(figsize=(12,5))
    sns.lineplot(data=df, x="timestamp", y="temperature", label="Temperature")
    sns.lineplot(data=df, x="timestamp", y="setpoint", label="Setpoint")
    plt.title("Forensic Timeline of Thermostat Activity")
    plt.xlabel("Timestamp")
    plt.ylabel("°C")
    plt.legend()
    plt.grid(True)
    plt.show()

# ============================== MAIN ==============================
def main():
    # Analyze static data
    forensics = SmartThermostatForensics('thermostat_data.json')
    forensics.analyze_temperature_history()
    forensics.analyze_schedule()
    forensics.analyze_user_interactions()
    forensics.analyze_network_traffic()

    print("\n📂 Extracting logs...")
    df = extract_thermostat_logs(LOG_FILE)
    if df is None or df.empty:
        print("❌ No data to analyze.")
        return

    analyze_thermostat_logs(df)

    print("\n🔍 Running anomaly detection...")
    anomalies = detect_anomalies(df, send_alerts=True)  # Set to False to skip alerts during testing

    if anomalies is not None and not anomalies.empty:
        generate_forensic_report(anomalies)

    generate_forensic_timeline(df)

main()


ModuleNotFoundError: No module named 'pandas'

In [ ]:
%pip install scapy

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.4 MB 8.3 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 4.3 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install twilio

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.3 MB/s  0:00:00

   -- -------------------------------------  1/17 [typing-extensions]
   ---- -----------------------------------  2/17 [PyJWT]
   ----------- ----------------------------  5/17 [idna]
   ---------------- -----------------------  7/17 [charset_normalizer]
   --------------------- ------------------  9/17 [attrs]
   --------------------- ------------------  9/17 [attrs]
   ---------------------------- ----------- 12/17 [requests]
   ---------------------------- ----------- 12/17 [requests]
   -------------------------------- ------- 14/17 [aiohttp]
   -------------------------------- ------- 14/17 [aiohttp]
   -------------------------------- ------- 14/17 [aiohttp]
   ------------------------------------- -- 16/17 [twilio]
   ------------------------------------- -- 16/1